<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/main/Adjoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adjoints

In [ ]:
import numpy as np
def forwardMatMult(A:np.ndarray,model:np.ndarray,data:np.ndarray):
    if A.shape[1]!=model.shape[0]:
        raise Exception("Model and array axis 1 don't agree in size")
        
    if A.shape[0]!=data.shape[0]:
        raise Exception("Data and array axis 2 don't agree in size")   
        
    for i2 in range(A.shape[0]):
        for i1 in range(A.shape[1]):
            data[i2]+=model[i1]*A[i2,i1]

            
def adjointMatMult(A:np.ndarray,model:np.ndarray,data:np.ndarray):
    if A.shape[1]!=model.shape[0]:
        raise Exception("Model and array axis 1 don't agree in size")
        
    if A.shape[0]!=data.shape[0]:
        raise Exception("Data and array axis 2 don't agree in size")   
        
    for i2 in range(A.shape[0]):
        for i1 in range(A.shape[1]):
        
            model[i1]+=data[i2]*A[i2,i1]

In [ ]:
import numpy as np



def forwardLinInterp(x_data:np.ndarray,o_model:float, d_model:float, 
                     model:np.ndarray,data:np.ndarray):
    if x_data.shape[0]!=data.shape[0]:
        raise Exception("Data and locations must agree in size")

    for id in  range(data.shape[0]):
      fpos=(x_data[id]-o_model)/d_model
      ipos=int(fpos)
      fpos=fpos-ipos
      data[id]+=model[ipos]*(1.-fpos) + model[ipos+1]*fpos
    

            
def adjointLinInterp(x_data:np.ndarray,o_model:float, d_model:float, 
                     model:np.ndarray,data:np.ndarray):
    if x_data.shape[0]!=data.shape[0]:
        raise Exception("Data and locations must agree in size")

    for id in  range(data.shape[0]):
      fpos=(x_data[id]-o_model)/d_model
      ipos=int(fpos)
      fpos=fpos-ipos
      #data[id]+=model[ipos]*(1.-fpos) + model[ipos+1]*fpos
      model[ipos]+=data[id]*(1.-fpos)
      model[ipos+1]+=data[id]*fpos



# Our first class!

In [ ]:
class myOperator:
  """ A generic operator class"""
  def __init__(self,domain,range):
    self._domain=domain
    self._range=range
  
  def forward(self,add:bool,model:np.ndarray,data:np.ndarray):
    raise Exception("Must override forward")

  def adjoint(self,add:bool,model:ndarray,data:ndarray):
    raise Exception("Must override adjoint")

  def checkDomainRange(self,mod:np.ndarray,dat:np.ndarray):
    if all mod.shape != self._domain.shape:
      raise Exception("mod and domain not the same")
    if all dat.shape != self._range.shape:
      raise Exception("dat and range not the same")
class Lin1D(myOperator):
  """Do linear interpolation 
  """

  def __init__(self,o_mod:float,d_mod:float,xpos:np.ndarray,model:np.ndarray,data:np.ndarray):
    super().__init(model,data)
    
    if xpos.shape[0]!=data.shape[0]:
      raise Exception("data and pos not the same size")
    
    self._xpos=xpos
    self._o=o_mod
    self._d=d_mod

    mx=self._o+self._d*(mod.shape[0]-1)
    for x in xpos:
      if x < self._o or x> mx:
        raise Exception("Data points outside model")

  def forward(self,add:bool,model:np.ndarray,data:np.ndarray):
      self.checkDomainRange(model,data)
      if not add:
        data.fill(0.)

      for id in  range(data.shape[0]):
        fpos=(x_data[id]-o_model)/d_model
        ipos=int(fpos)
        fpos=fpos-ipos
        data[id]+=model[ipos]*(1.-fpos) + model[ipos+1]*fpos
      

            
  def adjoint(self,add:bool,model:np.ndarray,data:np.ndarray):

      self.checkDomainRange(model,data)
      if not add:
        model.fill(0.)

      for id in  range(data.shape[0]):
        fpos=(x_data[id]-o_model)/d_model
        ipos=int(fpos)
        fpos=fpos-ipos
        #data[id]+=model[ipos]*(1.-fpos) + model[ipos+1]*fpos
        model[ipos]+=data[id]*(1.-fpos)
        model[ipos+1]+=data[id]*fpos


# Vector class


In [ ]:
import numpy as np
class myVector:

  def __init__(self,*arg,**kw):
    """Initialize a vector class 
    
        Two different ways

        ar - Numpy array

        shape - The size of the array

        Optional:

          spaceOnly - Whether or not to allocate space for the array (default False)
    
    """

    self._spaceOnly=False
    if "spaceOnly" in kw:
      if not isinstance(kw["spaceOnly"],bool):
        raise Exception("spaceOnly should be a bool")
      self._spaceOnly=kw["spaceOnly"]
    if isinstance(arg[0],np.ndarray):
      self._ar=np.copy(arg[0])
      self._shape=list(arg[0].shape)
    elif isinstance(arg[0],list):
      self._ar=np.ndarray(arg[0])
      self._shape=arg[0]
    else:
      raise Exception(f"Arg must be a ndarray or list (size) type is {type(arg[0])}")
  
  def checkSame(self,vec:np.ndarray):
    """Check to make sure vectors match"""
    if list(self._ar.shape)!=list(vec._ar.shape):
      return False
    return True

  def validOp(self):
    """Check to make sure class is allocated"""
    if self._spaceOnly:
      raise Exception("Asked to use a vector when not allocated")
  
  def zero(self):
    "Zero vector"
    self.validOp()
    self._ar.fill(0)
  
  def scale(self,sc:float):
    """Scale a vector"""
    self.validOp()
    self._ar=self._ar*sc

  def scaleAdd(self,vec:myVector,sc1:float=1,sc2:float=1.):
    """ vec=self*sc1+vec*sc2"""
    self.validOp()
    vec.validOp()
    self.checkSame(vec)
    self._ar=self._ar*sc1+vec._ar*sc2

  def clone(self):
    """Make a copy of the vector"""
    if not self._spaceOnly: myVector(self._ar)
    return myVector(self._shape)
  
  def cloneSpace(self):
    """Make a copy of just the vector space"""
    return myVector(self._shape,spaceOnly=True)

  def rand(self):
    """Fill with a random number"""
    self._arr=np.random.rand(self._shape)


    

In [ ]:
class myOperator:
  """ A generic operator class"""
  def __init__(self,domain,range):
    self._domain=domain.cloneSpace()
    self._range=range.cloneSpace()
  
  def forward(self,add:bool,model:np.ndarray,data:np.ndarray):
    raise Exception("Must override forward")

  def adjoint(self,add:bool,model:np.ndarray,data:np.ndarray):
    raise Exception("Must override adjoint")

  def checkDomainRange(self,mod:np.ndarray,dat:np.ndarray):
    self._domain.checkSame(mod)
    self._range.checkSame(dat)


In [ ]:
class Lin1D(myOperator):
  """Do linear interpolation 
  """

  def __init__(self,o_mod:float,d_mod:float,xpos:np.ndarray,model:myVector,data:myVector):
    super().__init__(model,data)
    
    if xpos.shape[0]!=data._shape[0]:
      raise Exception(f"xpos and data not the same size {xpos.shape[0]} {data._shape[0]}")
    
    self._xpos=xpos
    self._o=o_mod
    self._d=d_mod

    mx=self._o+self._d*(mod._shape[0]-1)
    for x in xpos:
      if x < self._o or x>= mx:
        raise Exception(f"Data points outside model val {x} min {self._o} max {mx}")

  def forward(self,add:bool,model:myVector,data:myVector):
      self.checkDomainRange(model,data)
      if not add:
        data.zerp()

      for id in  range(data._shape[0]):
        fpos=(self._xpos[id]-self._o)/self._d
        ipos=int(fpos)
        fpos=fpos-ipos
        data._ar[id]+=model._ar[ipos]*(1.-fpos) + model._ar[ipos+1]*fpos
      

            
  def adjoint(self,add:bool,model:myVector,data:myVector):

      self.checkDomainRange(model,data)
      if not add:
        model.zero()

      for id in  range(data._shape[0]):
        fpos=(self._xpos[id]-self._o)/self._d
        ipos=int(fpos)
        fpos=fpos-ipos
        model._ar[ipos]+=data._ar[id]*(1.-fpos)
        model._ar[ipos+1]+=data._ar[id]*fpos

In [ ]:
x=np.array([1.4,2.2,3.1,4.4])
y=np.array([1,2,3,4])
m=np.ndarray((10))

dat=myVector(y)
mod=myVector(m)

op=Lin1D(0.,.5,x,mod,dat)
op.adjoint(False,mod,dat)
print(mod._ar)

[0.  0.  0.2 0.8 1.2 0.8 2.4 0.6 0.8 3.2]
